# Procedimientos Almacenados  2 - PL/pgSQL

Importamos la librería para ejecutar código SQL en este jupyter notebook antes de empezar:

In [ ]:
%load_ext sql

Debes modificar la siguiente celda para que tenga los datos correctos para conectarse la instancia de PostgreSQL que corre en tu computador.

``postgresql://<usuario_de_postgres>:<pass_de_postgres>@localhost/<nombre_db>``

In [ ]:
%%sql 
postgresql://adriansotosuarez:@localhost/almacenados

### Agradecimientos

Este notebook fue escrito principalmente por el ayudante **Marcelo Saldías González**.

# 1. Trucos Avanzados

¿Puedes adivinar que computa esta función?

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION fib (
numero INTEGER
) RETURNS INTEGER AS $$
BEGIN
IF numero < 2 THEN
RETURN numero;
END IF;
RETURN fib(numero - 2) + fib(numero - 1);
END;
$$ LANGUAGE plpgsql;

In [ ]:
%sql SELECT fib(10);

Exacto! Son los números de Fibonacci! Esto no tiene nada que ver con las bases de datos, dices. Pero ahora trata de ejecutar esta función en tu computador. Utiliza la celda anterior para probar con ```n = 10```, ```n = 20```, y ```n = 30```... Se demora muchísimo, y si te acuerdas de introducción a la programación, tiene toda la razón de demorarse, porque nuestro algoritmo recursivo está mal escrito. Por ejemplo, para n = 30 tenemos que llamar a ```fib(28)``` y ```fib(29)```, pero ambas requieren el cómputo de ```fib(27)```, y por lo tanto las ejecuciones de ```fib``` para números menores terminan ejecutándose miles de veces.

Bueno, aprovechémonos de que tenemos una base de datos, y tratemos de guardar en una tabla temporal los resultados de fibonacci. No debería tomar mucho espacio.

En la siguiente celda, crea una relación

`Fib_cache(num INTEGER, fib INTEGER)`

In [ ]:
%%sql
DROP TABLE IF EXISTS Fib_cache;
CREATE TABLE Fib_cache(num integer, fib integer, PRIMARY KEY(num));

La relación anterior almacenará cada número ```num``` junto al número Fibonacci correspondiente a ```num``` (es decir, ```fib(num)```).

Ahora observa

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION fib_cacheando(
  numero INTEGER
) RETURNS INTEGER AS $$
DECLARE
ret INTEGER;
BEGIN
IF numero < 2 THEN
    RETURN numero;
END IF;
SELECT INTO ret fib
FROM fib_cache
WHERE num = numero;
IF ret IS NULL THEN
    ret := fib_cacheando(numero - 2) + fib_cacheando(numero - 1);
    INSERT INTO fib_cache (num, fib)
    VALUES (numero, ret);
END IF;
RETURN ret;
END;
$$ language plpgsql;

### ¿Que crees que esta pasando? Vamos paso por paso:

```SQL
CREATE OR REPLACE FUNCTION fib_cacheando(
  numero INTEGER
) RETURNS INTEGER AS $$
DECLARE
ret INTEGER;
BEGIN
IF numero < 2 THEN
    RETURN numero;
END IF;
```

Todo bien hasta acá, tomamos los casos bases 0 y 1

```SQL
SELECT INTO ret fib
FROM fib_cache
WHERE num = numero;
```

Usamos ```SELECT INTO <variable> <atributo> FROM...```  para guardar en ```<variable>``` el valor de la consulta ```SELECT <atributo> FROM...```. En este caso, buscamos en la tabla ```fib_cache``` si acaso tenemos la tupla del número de Fibonacci correspondiente a ```num```.

```SQL
IF ret IS NULL THEN
    ret := fib_cacheando(numero - 2) + fib_cacheando(numero - 1);
    INSERT INTO fib_cache (num, fib)
    VALUES (numero, ret);
END IF;
```

Si ```ret``` es nulo, significa que no tenemos esevalor, y hay que obtenerlo utilizando una llamada recursiva. Si ```ret``` no es nulo, significa que ya lo tenemos, y no tenemos que hacer nada más.

```sql
RETURN ret;
END;
$$ LANGUAGE plpsgsql
```

Finalmente retornamos el valor de ```ret```. Prueba ahora calcular ```fib_cacheando(30)```. ¡Mira lo rápido que funciona ahora!

In [ ]:
%sql SELECT fib_cacheando(30);

# 2. Retornando Tablas

En la siguiente celda, crea la sigiuente relación:

Vuelo(**ciudad_origen:varchar, ciudad_destino:varchar**, horas:integer)

In [ ]:
%%sql
DROP TABLE IF EXISTS Vuelo;
CREATE TABLE Vuelo (ciudad_origen varchar, ciudad_destino varchar, horas integer, PRIMARY KEY(ciudad_origen, ciudad_destino));

La idea de esta relación es que una tupla `(ciudad origen, ciudad destino, N)` indique que existe al menos un vuelo directo desde la ciudad origen a la ciudad destino, y que el vuelo más corto demora $N$ horas. Con la siguiente celda, llenaremos esta tabla con unas cuantas tuplas para ir probando las funciones que debes hacer.

In [ ]:
%%sql
INSERT INTO Vuelo VALUES ('Santiago', 'Concepción', 2);
INSERT INTO Vuelo VALUES ('Santiago', 'Seoul', 23);
INSERT INTO Vuelo VALUES ('Concepción', 'Dallas', 24);
INSERT INTO Vuelo VALUES ('Santiago', 'Shanghai', 10);
INSERT INTO Vuelo VALUES ('New York', 'Los Angeles', 12);
INSERT INTO Vuelo VALUES ('New York', 'London', 20);
INSERT INTO Vuelo VALUES ('Dallas', 'Houston', 10);
INSERT INTO Vuelo VALUES ('Dallas', 'Toronto', 15);
INSERT INTO Vuelo VALUES ('Toronto', 'Chengdu', 12);
INSERT INTO Vuelo VALUES ('Toronto', 'Huangzhou', 20);

Queremos crear una función que reciba una ciudad C y retorne una tabla con todas las ciudades en las que existe un vuelo directo desde C, junto con el tiempo. Para esto, tenemos que decirle a la función que debe retornar una tabla (si, esto se puede hacer en SQL, pero lo hacemos acá porque es un buen ejercicio para empezar).

Veamos primero como retornar una tabla cualquiera:

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION
retornar_vuelos ()
RETURNS TABLE (ciudad_origen varchar(50), ciudad_destino varchar(50), 
              horas integer) AS $$
BEGIN
RETURN QUERY SELECT * FROM Vuelo;
RETURN;
END;
$$ language plpgsql

Nada muy impactante... lo único que hay que tomar en cuenta es que ```RETURN QUERY``` no retorna una función, solo asigna el valor de retorno a la tabla en cuestión (por eso retornamos explícitamente después).

Una vez que cargues esta funcion en PostgreSQL, prueba llamándola como hemos hecho hasta ahora:

In [ ]:
%sql SELECT retornar_vuelos();

¿Algún Problema? ¿No se supone que retornaba una tabla? Efectivamente, y por eso es que si queremos que la seleccion haga explícito todos sus atributos, tenemos que consultar lo que retorna la función como si fuera una tabla!

In [ ]:
%sql SELECT * from retornar_vuelos();

# 2.1 SQL Dinámico

¿Qué pasa cuando quiero que mi consulta se vea modificada de acuerdo al input de la función?

En ese caso tenemos dos opciones:

+ Generar la consulta como una concatenación de strings (alternativa mala)
+ Usar SQL Dinámico (alternativa decente)

Por qué es mejor SQL dinámico lo discutiremos en clases. Pero veamos en que consiste.

La idea de SQL dinámico es "preparar" una consulta antes de saber los parámetros, y una vez que tengamos el valor de los parámetros, ejecutar la consulta, mediante el comando ```EXECUTE```.

Veamos el ejemplo de los vuelos directos:

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION
vuelos_directos (c_origen varchar)
RETURNS TABLE (ciudad_destino varchar(50), horas integer) AS $$
BEGIN
RETURN QUERY EXECUTE 'SELECT ciudad_destino, horas
        FROM VUELO
        WHERE ciudad_origen = $1'
    USING c_origen;
RETURN;
END;
$$ language plpgsql;


La idea es almacenar en variables `($1, $2, etc)` los valores que necesitamos, y luego especificar las variables de donde sacamos estos valores con ```USING```.

In [ ]:
%sql SELECT * FROM vuelos_directos('Dallas');

# 2.2 Ejercicio

Imagina ahora que tienes una instancia de la tabla Vuelo con miles de datos, y te interesan todos los lugares a los que puedes llegar volando desde Santiago, sin importar el número de escalas. ¿Puedes pensar en un algoritmo para implementar esto usando una funcion?

Escríbelo como una función en PLpgSQL. Puede que te sea útil pensar en como reutilizábamos una base de datos temporal para computar los números de Fibonacci.

In [ ]:
%%sql
#Esribe tu respuesta aqui!